In [2]:
import pandas as pd

# Load the data
df_1963_sample = pd.read_excel('/home/samirk08/UROP_SPRING_2024/1963/1963-1000Sample.xlsx')
tsus_hts_df = pd.read_excel('/home/samirk08/UROP_SPRING_2024/UROP IAP 2024/Original Databases/HTS_TSUS.xlsx')

# Helper function to sanitize the 'Item' values
def sanitize_item(item):
    try:
        # Attempt to remove decimal and convert to int
        return int(float(item) * 100)
    except ValueError:
        # Handle cases with non-numeric text
        # Extract the numeric part before the space or '--' if present
        numeric_part = item.split()[0].split('--')[0]
        try:
            # Now try to convert this cleaned numeric part to an integer
            return int(float(numeric_part) * 100)
        except ValueError:
            # If still not convertible, return None or a specific flag
            return None

# Apply the sanitization function to the 'Item' column
df_1963_sample['Item'] = df_1963_sample['Item'].apply(sanitize_item)

# Step 2 and 3: Find the closest TSUS match and retrieve corresponding HTS code
def find_closest_tsus_and_hts(item_code):
    # Ensure item_code is not None
    if item_code is None:
        return None
    # Convert TSUS codes to string for matching
    tsus_hts_df['TSUS_str'] = tsus_hts_df['TSUS'].astype(str)
    # Get the length of the item_code to use in matching
    item_code_len = len(str(item_code))
    # Find the closest TSUS match by matching the first n characters where n is the length of the item_code
    closest_tsus = tsus_hts_df[tsus_hts_df['TSUS_str'].str.startswith(str(item_code))]['TSUS'].values
    if closest_tsus.size > 0:
        # If there are multiple matches, choose the first one
        tsus_code = closest_tsus[0]
        # Get the corresponding HTS code for the closest TSUS match
        hts_code = tsus_hts_df[tsus_hts_df['TSUS'] == tsus_code]['HTS'].values[0]
        return hts_code
    else:
        # If no match found, return None or a placeholder value
        return None

# Apply the function to each 'Item' in the DataFrame
df_1963_sample['HTS Code'] = df_1963_sample['Item'].apply(find_closest_tsus_and_hts)

# Step 4: Save the updated DataFrame to a new Excel file
output_path = '/home/samirk08/UROP_SPRING_2024/1963/updated_1963Sample_with_hts.xlsx'
df_1963_sample.to_excel(output_path, index=False)
